In [1]:
# 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 導入所需庫
import pandas as pd

# 指定 Google Drive 中資料夾的路徑
data_path = '/content/drive/MyDrive/Flightdelays/'  # 修改為你的資料夾路徑

# 讀取數據
train_data = pd.read_csv(data_path + 'flight_delays_train.csv')
test_data = pd.read_csv(data_path + 'flight_delays_test.csv')

# 確認數據讀取成功
print("訓練集：")
print(train_data.head())

print("\n測試集：")
print(test_data.head())

Mounted at /content/drive
訓練集：
  Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0   c-8       c-21       c-7     1934            AA    ATL  DFW       732   
1   c-4       c-20       c-3     1548            US    PIT  MCO       834   
2   c-9        c-2       c-5     1422            XE    RDU  CLE       416   
3  c-11       c-25       c-6     1015            OO    DEN  MEM       872   
4  c-10        c-7       c-6     1828            WN    MDW  OMA       423   

  dep_delayed_15min  
0                 N  
1                 N  
2                 N  
3                 N  
4                 Y  

測試集：
  Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance
0   c-7       c-25       c-3      615            YV    MRY  PHX       598
1   c-4       c-17       c-2      739            WN    LAS  HOU      1235
2  c-12        c-2       c-7      651            MQ    GSP  ORD       577
3   c-3       c-25       c-7     1614            WN    BWI  MHT       37

In [11]:
# 掛載 Google Drive 並載入數據
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import numpy as np

# 掛載 Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Flightdelays/'

# 讀取數據
train_data = pd.read_csv(data_path + 'flight_delays_train.csv')
test_data = pd.read_csv(data_path + 'flight_delays_test.csv')

# 預處理數據
le = LabelEncoder()
train_data['dep_delayed_15min'] = le.fit_transform(train_data['dep_delayed_15min'])
categorical_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

# Handle unseen labels using a single LabelEncoder for both train and test data
for col in categorical_cols:
    # Fit on the combined unique values from both train and test data
    all_values = pd.concat([train_data[col], test_data[col]]).unique()
    le.fit(all_values)
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

X = train_data.drop(columns=['dep_delayed_15min'])
y = train_data['dep_delayed_15min']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練 XGBoost 模型
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)
# 訓練模型！ This is the crucial line you were missing.
xgb_model.fit(X_train, y_train)

# 評估模型
y_train_probs = xgb_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_train_probs)
print(f"訓練集 ROC AUC Score: {roc_auc}")

# 預測測試集的延誤概率
y_probs = xgb_model.predict_proba(test_data)[:, 1]

# 建立提交文件
submission = pd.DataFrame({
    'id': np.arange(len(y_probs)),  # 創建 ID 列
    'dep_delayed_15min': y_probs   # 延誤的預測概率
})

# 保存提交文件到 Google Drive 的 Flightdelays 資料夾
submission_path = '/content/drive/MyDrive/Flightdelays/sample_submission_xgb.csv'
submission.to_csv(submission_path, index=False)

# 顯示提交文件內容
print("Sample Submission:")
print(submission.head())

print(f"提交文件已儲存在: {submission_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:41:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


訓練集 ROC AUC Score: 0.7398914391170838
Sample Submission:
   id  dep_delayed_15min
0   0           0.019304
1   1           0.038770
2   2           0.036690
3   3           0.210994
4   4           0.152942
提交文件已儲存在: /content/drive/MyDrive/Flightdelays/sample_submission_xgb.csv


In [12]:
# 掛載 Google Drive 並載入數據
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import numpy as np

# 掛載 Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Flightdelays/'

# 讀取數據
train_data = pd.read_csv(data_path + 'flight_delays_train.csv')
test_data = pd.read_csv(data_path + 'flight_delays_test.csv')

# 預處理數據
le = LabelEncoder()
train_data['dep_delayed_15min'] = le.fit_transform(train_data['dep_delayed_15min'])
categorical_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

# Handle unseen labels using a single LabelEncoder for both train and test data
for col in categorical_cols:
    # Fit on the combined unique values from both train and test data
    all_values = pd.concat([train_data[col], test_data[col]]).unique()
    le.fit(all_values)
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

X = train_data.drop(columns=['dep_delayed_15min'])
y = train_data['dep_delayed_15min']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練 XGBoost 模型
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=500,  # 增加弱學習器數量
    learning_rate=0.05,  # 減小學習率
    max_depth=8,  # 增加樹深度
    subsample=0.8,  # 使用80%的數據子集
    colsample_bytree=0.8,  # 使用80%的特徵子集
    random_state=42
)
# 訓練模型！ This is the crucial line you were missing.
xgb_model.fit(X_train, y_train)

# 評估模型
y_train_probs = xgb_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_train_probs)
print(f"訓練集 ROC AUC Score: {roc_auc}")

# 預測測試集的延誤概率
y_probs = xgb_model.predict_proba(test_data)[:, 1]

# 建立提交文件
submission = pd.DataFrame({
    'id': np.arange(len(y_probs)),  # 創建 ID 列
    'dep_delayed_15min': y_probs   # 延誤的預測概率
})

# 保存提交文件到 Google Drive 的 Flightdelays 資料夾
submission_path = '/content/drive/MyDrive/Flightdelays/sample_submission_xgb.csv'
submission.to_csv(submission_path, index=False)

# 顯示提交文件內容
print("Sample Submission:")
print(submission.head())

print(f"提交文件已儲存在: {submission_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:45:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


訓練集 ROC AUC Score: 0.7511503145314558
Sample Submission:
   id  dep_delayed_15min
0   0           0.026111
1   1           0.038524
2   2           0.053435
3   3           0.129550
4   4           0.145718
提交文件已儲存在: /content/drive/MyDrive/Flightdelays/sample_submission_xgb.csv


#決策樹

In [13]:
# 掛載 Google Drive 並載入數據
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# 掛載 Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Flightdelays/'

# 讀取數據
train_data = pd.read_csv(data_path + 'flight_delays_train.csv')
test_data = pd.read_csv(data_path + 'flight_delays_test.csv')

# 預處理數據
le = LabelEncoder()
train_data['dep_delayed_15min'] = le.fit_transform(train_data['dep_delayed_15min'])
categorical_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

# Handle unseen labels using a single LabelEncoder for both train and test data
for col in categorical_cols:
    # Fit on the combined unique values from both train and test data
    all_values = pd.concat([train_data[col], test_data[col]]).unique()
    le.fit(all_values)
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

X = train_data.drop(columns=['dep_delayed_15min'])
y = train_data['dep_delayed_15min']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練決策樹模型
dt_model = DecisionTreeClassifier(
    criterion='gini',  # 或 'entropy' 可選
    max_depth=10,  # 控制樹的深度
    min_samples_split=10,  # 分割內部節點所需的最小樣本數
    random_state=42
)

dt_model.fit(X_train, y_train)

# 評估模型
y_train_probs = dt_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_train_probs)
print(f"訓練集 ROC AUC Score: {roc_auc}")

# 預測測試集的延誤概率
y_probs = dt_model.predict_proba(test_data)[:, 1]

# 建立提交文件
submission = pd.DataFrame({
    'id': np.arange(len(y_probs)),  # 創建 ID 列
    'dep_delayed_15min': y_probs   # 延誤的預測概率
})

# 保存提交文件到 Google Drive 的 Flightdelays 資料夾
submission_path = '/content/drive/MyDrive/Flightdelays/sample_submission_dt_tree.csv'
submission.to_csv(submission_path, index=False)

# 顯示提交文件內容
print("Sample Submission:")
print(submission.head())

print(f"提交文件已儲存在: {submission_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
訓練集 ROC AUC Score: 0.6943567917719723
Sample Submission:
   id  dep_delayed_15min
0   0           0.032258
1   1           0.046948
2   2           0.031050
3   3           0.150000
4   4           0.185590
提交文件已儲存在: /content/drive/MyDrive/Flightdelays/sample_submission_dt_tree.csv
